In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [3]:

mean = (0.4914, 0.4822, 0.4465)
std  = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

trainset = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=transform_train
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2
)

testset = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=transform_test
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2
)


In [27]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels,
                               kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels,
                               kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut (projection) if shape changes (stride != 1 or channels differ)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out


In [28]:
class ResNet20(nn.Module):
    def __init__(self, block=BasicBlock, num_classes=10):
        super(ResNet20, self).__init__()
        self.in_channels = 16

        # Initial conv: 3x3, 16 filters
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        # Each "layer" is a sequence of blocks
        self.layer1 = self._make_layer(block, out_channels=16, blocks=3, stride=1)
        self.layer2 = self._make_layer(block, out_channels=32, blocks=3, stride=2)
        self.layer3 = self._make_layer(block, out_channels=64, blocks=3, stride=2)

        # Global average pool and final classification layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride):
        """Create a stage of 'blocks' residual blocks."""
        strides = [stride] + [1] * (blocks - 1)  
        layers = []
        for s in strides:
            layers.append(block(self.in_channels, out_channels, s))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out


In [29]:
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc


In [30]:
model = ResNet20().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

In [31]:
#transformations for metamorphic testing
transformations = [
    transforms.RandomRotation(degrees=10),  # Small rotation
    transforms.ColorJitter(brightness=0.2),  # Slight brightness change
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # Small translation
]

In [32]:
reject_diffs = []

# Evaluate reject differential over the test set
for i, (images, labels) in enumerate(tqdm(testloader)):
    
    images = images.to(device)

    # Original prediction
    with torch.no_grad():
        original_output = model(images)
        original_prob = F.softmax(original_output, dim=1)               # shape [batch_size, 10]
        original_confidence, original_pred = torch.max(original_prob, 1) # shape [batch_size]

    # For metamorphic transform
    for tf in transformations:
       
        transformed_images = tf(images.cpu()).to(device)

        with torch.no_grad():
            transformed_output = model(transformed_images)
            transformed_prob = F.softmax(transformed_output, dim=1)
            transformed_confidence, transformed_pred = torch.max(transformed_prob, 1)

        # Compute absolute difference in confidence for each sample in the batch
    
        diff_tensor = torch.abs(original_confidence - transformed_confidence)

        # Convert to a single scalar by taking the mean over the batch
        diff_mean = diff_tensor.mean().item()

       
        reject_diffs.append(diff_mean)

       
        print(f"Batch {i} | Transform: {tf.__class__.__name__} | "
              f"Reject Differential (mean): {diff_mean:.4f}")


print("Number of recorded reject differentials:", len(reject_diffs))
print("Example of first few reject_diffs:", reject_diffs[:5])


  4%|██████▌                                                                                                                                                              | 4/100 [00:00<00:08, 10.82it/s]

Batch 0 | Transform: RandomRotation | Reject Differential (mean): 0.0000
Batch 0 | Transform: ColorJitter | Reject Differential (mean): 0.0207
Batch 0 | Transform: RandomAffine | Reject Differential (mean): 0.0151
Batch 1 | Transform: RandomRotation | Reject Differential (mean): 0.0111
Batch 1 | Transform: ColorJitter | Reject Differential (mean): 0.0219
Batch 1 | Transform: RandomAffine | Reject Differential (mean): 0.0148
Batch 2 | Transform: RandomRotation | Reject Differential (mean): 0.0129
Batch 2 | Transform: ColorJitter | Reject Differential (mean): 0.0194
Batch 2 | Transform: RandomAffine | Reject Differential (mean): 0.0157
Batch 3 | Transform: RandomRotation | Reject Differential (mean): 0.0135
Batch 3 | Transform: ColorJitter | Reject Differential (mean): 0.0192
Batch 3 | Transform: RandomAffine | Reject Differential (mean): 0.0146
Batch 4 | Transform: RandomRotation | Reject Differential (mean): 0.0142
Batch 4 | Transform: ColorJitter | Reject Differential (mean): 0.0193


  8%|█████████████▏                                                                                                                                                       | 8/100 [00:00<00:05, 17.15it/s]

Batch 4 | Transform: RandomAffine | Reject Differential (mean): 0.0145
Batch 5 | Transform: RandomRotation | Reject Differential (mean): 0.0142
Batch 5 | Transform: ColorJitter | Reject Differential (mean): 0.0191
Batch 5 | Transform: RandomAffine | Reject Differential (mean): 0.0167
Batch 6 | Transform: RandomRotation | Reject Differential (mean): 0.0128
Batch 6 | Transform: ColorJitter | Reject Differential (mean): 0.0237
Batch 6 | Transform: RandomAffine | Reject Differential (mean): 0.0146
Batch 7 | Transform: RandomRotation | Reject Differential (mean): 0.0149
Batch 7 | Transform: ColorJitter | Reject Differential (mean): 0.0220
Batch 7 | Transform: RandomAffine | Reject Differential (mean): 0.0116
Batch 8 | Transform: RandomRotation | Reject Differential (mean): 0.0136
Batch 8 | Transform: ColorJitter | Reject Differential (mean): 0.0202
Batch 8 | Transform: RandomAffine | Reject Differential (mean): 0.0140
Batch 9 | Transform: RandomRotation | Reject Differential (mean): 0.0121


 14%|██████████████████████▉                                                                                                                                             | 14/100 [00:00<00:03, 22.97it/s]

...

Batch 78 | Transform: RandomRotation | Reject Differential (mean): 0.0125
Batch 78 | Transform: ColorJitter | Reject Differential (mean): 0.0213
Batch 78 | Transform: RandomAffine | Reject Differential (mean): 0.0158
Batch 79 | Transform: RandomRotation | Reject Differential (mean): 0.0134
Batch 79 | Transform: ColorJitter | Reject Differential (mean): 0.0177
Batch 79 | Transform: RandomAffine | Reject Differential (mean): 0.0154
Batch 80 | Transform: RandomRotation | Reject Differential (mean): 0.0101
Batch 80 | Transform: ColorJitter | Reject Differential (mean): 0.0182
Batch 80 | Transform: RandomAffine | Reject Differential (mean): 0.0127
Batch 81 | Transform: RandomRotation | Reject Differential (mean): 0.0120
Batch 81 | Transform: ColorJitter | Reject Differential (mean): 0.0216
Batch 81 | Transform: RandomAffine | Reject Differential (mean): 0.0136
Batch 82 | Transform: RandomRotation | Reject Differential (mean): 0.0115
Batch 82 | Transform: ColorJitter | Reject Differential (m

 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊        | 95/100 [00:03<00:00, 41.16it/s]

Batch 87 | Transform: RandomRotation | Reject Differential (mean): 0.0129
Batch 87 | Transform: ColorJitter | Reject Differential (mean): 0.0193
Batch 87 | Transform: RandomAffine | Reject Differential (mean): 0.0129
Batch 88 | Transform: RandomRotation | Reject Differential (mean): 0.0158
Batch 88 | Transform: ColorJitter | Reject Differential (mean): 0.0236
Batch 88 | Transform: RandomAffine | Reject Differential (mean): 0.0144
Batch 89 | Transform: RandomRotation | Reject Differential (mean): 0.0120
Batch 89 | Transform: ColorJitter | Reject Differential (mean): 0.0224
Batch 89 | Transform: RandomAffine | Reject Differential (mean): 0.0117
Batch 90 | Transform: RandomRotation | Reject Differential (mean): 0.0128
Batch 90 | Transform: ColorJitter | Reject Differential (mean): 0.0191
Batch 90 | Transform: RandomAffine | Reject Differential (mean): 0.0122
Batch 91 | Transform: RandomRotation | Reject Differential (mean): 0.0119
Batch 91 | Transform: ColorJitter | Reject Differential (m

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 30.29it/s]

Batch 95 | Transform: RandomAffine | Reject Differential (mean): 0.0133
Batch 96 | Transform: RandomRotation | Reject Differential (mean): 0.0074
Batch 96 | Transform: ColorJitter | Reject Differential (mean): 0.0204
Batch 96 | Transform: RandomAffine | Reject Differential (mean): 0.0188
Batch 97 | Transform: RandomRotation | Reject Differential (mean): 0.0136
Batch 97 | Transform: ColorJitter | Reject Differential (mean): 0.0202
Batch 97 | Transform: RandomAffine | Reject Differential (mean): 0.0150
Batch 98 | Transform: RandomRotation | Reject Differential (mean): 0.0102
Batch 98 | Transform: ColorJitter | Reject Differential (mean): 0.0202
Batch 98 | Transform: RandomAffine | Reject Differential (mean): 0.0136
Batch 99 | Transform: RandomRotation | Reject Differential (mean): 0.0083
Batch 99 | Transform: ColorJitter | Reject Differential (mean): 0.0235
Batch 99 | Transform: RandomAffine | Reject Differential (mean): 0.0000
Number of recorded reject differentials: 300
Example of firs

In [33]:
train_losses = []
test_losses = []

num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, trainloader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, testloader, criterion)

    # Append losses to track
    train_losses.append(train_loss)
    test_losses.append(test_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Test  Loss: {test_loss:.4f}, Test  Acc: {test_acc:.2f}%")

Epoch [1/10]
  Train Loss: 1.6972, Train Acc: 36.04%
  Test  Loss: 1.5370, Test  Acc: 43.60%
Epoch [2/10]
  Train Loss: 1.2336, Train Acc: 55.22%
  Test  Loss: 1.3818, Test  Acc: 55.02%
Epoch [3/10]
  Train Loss: 0.9726, Train Acc: 65.30%
  Test  Loss: 1.0759, Test  Acc: 63.55%
Epoch [4/10]
  Train Loss: 0.7995, Train Acc: 71.76%
  Test  Loss: 0.9595, Test  Acc: 68.67%
Epoch [5/10]
  Train Loss: 0.6989, Train Acc: 75.60%
  Test  Loss: 0.7330, Test  Acc: 75.12%
Epoch [6/10]
  Train Loss: 0.6304, Train Acc: 78.02%
  Test  Loss: 0.7427, Test  Acc: 74.58%
Epoch [7/10]
  Train Loss: 0.5719, Train Acc: 80.19%
  Test  Loss: 0.7963, Test  Acc: 73.50%
Epoch [8/10]
  Train Loss: 0.5357, Train Acc: 81.52%
  Test  Loss: 0.7665, Test  Acc: 75.32%
Epoch [9/10]
  Train Loss: 0.5039, Train Acc: 82.48%
  Test  Loss: 0.5829, Test  Acc: 80.47%
Epoch [10/10]
  Train Loss: 0.4822, Train Acc: 83.16%
  Test  Loss: 0.6081, Test  Acc: 80.13%


In [34]:
def validate(model, testloader, device, criterion):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    all_preds = []
    all_targets = []
    
    # Switch off gradient tracking for validation
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            
            # Forward pass
            output = model(data)
            loss = criterion(output, target)
            
            # Accumulate loss * batch_size for accurate average
            val_running_loss += loss.item() * data.size(0)
            
            # Predictions
            _, preds = torch.max(output, 1)
            val_running_correct += (preds == target).sum().item()
            
            # Store for metrics
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    # Calculate average loss
    total_samples = len(testloader.dataset)
    val_loss = val_running_loss / total_samples
    
    # Calculate accuracy
    val_accuracy = 100.0 * val_running_correct / total_samples
    
    # Calculate precision, recall, F1 (weighted or macro—your choice)
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall    = recall_score(all_targets, all_preds, average='weighted')
    f1        = f1_score(all_targets, all_preds, average='weighted')
    
    return val_loss, val_accuracy, precision, recall, f1


In [35]:
model_save_path = 'resnet20_RejDiff_cifar10.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to resnet20_RejDiff_cifar10.pth


In [36]:
model.load_state_dict(torch.load(model_save_path))
mode = model.to(device)

/tmp/ipykernel_320522/1717748095.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))


In [37]:
final_val_loss, final_val_accuracy, final_precision, final_recall, final_f1 = validate( model,  testloader, device=device, criterion=criterion)

print(f"\nFinal Test Results - Loss: {final_val_loss:.4f}, "f"Accuracy: {final_val_accuracy:.2f}%, "f"Precision: {final_precision:.2f}, "f"Recall: {final_recall:.2f}, "f"F1 Score: {final_f1:.2f}")


Final Test Results - Loss: 0.6081, Accuracy: 80.13%, Precision: 0.81, Recall: 0.80, F1 Score: 0.80
